In [ ]:
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path

initial_path = 'C:/Users/Mateus Santos Rochas/Desktop/Estudos/07. Doutorado - Matemática aplicada/Pesquisa/Dados R2'
bronze_path = os.path.join(initial_path,'1. bronze')
prata_path = os.path.join(initial_path,'2. prata')
ouro_path = os.path.join(initial_path,'3. ouro')

conn = duckdb.connect('r2_prata.duckdb')
conn.execute("ATTACH 'r2_ouro.duckdb' AS r2_ouro")


def sql(query,conn=conn):
    return conn.execute(query).fetch_df()
    
show_tables_prata_query = """ SELECT DISTINCT table_name FROM information_schema.tables WHERE table_catalog = 'r2_prata' """
show_tables_ouro_query = """ SELECT DISTINCT table_name FROM information_schema.tables WHERE table_catalog = 'r2_ouro' """

# Funções

## Importar ABT

### import_abt

In [2]:
def import_abt(satelite_list,n_estacoes_vizinhas=4,min_year_training=2010,max_prec_filter=51):
    satelite_columns_str_dict = {
        'AgCFSR':
            """ ,vl_precipitacao_AgCFSR
                ,vl_umidade_relativa_maxima_AgCFSR
                ,vl_radiacao_solar_AgCFSR
                ,vl_temperatura_media_AgCFSR
                ,vl_temperatura_maxima_AgCFSR
                ,vl_temperatura_minima_AgCFSR
                ,vl_vento_velocidade_AgCFSR""",
        'POWER':
            """ ,vl_radiacao_solar_ceu_total_POWER
                ,vl_precipitacao_total_corrigido_POWER
                ,vl_pressao_superficie_POWER
                ,vl_umidade_relativa_2m_POWER
                ,vl_temperatura_maxima_2m_POWER
                ,vl_temperatura_minima_2m_POWER
                ,vl_temperatura_media_2m_POWER
                ,vl_temperatura_orvalho_2m_POWER
                ,vl_direcao_vento_2m_POWER
                ,vl_direcao_vento_10m_POWER
                ,vl_maxima_vento_2m_POWER
                ,vl_ws2m_POWER
                ,vl_maxima_vento_10m_POWER
                ,vl_vento_10m_POWER""",
        'GPM_Late_Run':
            """ ,vl_precipitacao_GPM_Late_Run""",
        'GPM_Final_Run':
            """ ,vl_precipitacao_GPM_Final_Run""",
        'GL':
            """ ,vl_irradiancia_GL""",
        'CPC':
            """ ,vl_precipitacao_CPC""",
        'TRMM':
            """ ,vl_precipitacao_TRMM""",
        'CHIRPS':
            """ ,vl_precipitacao_CHIRPS""",
        'PERSIANN_CDR':
            """ ,vl_precipitacao_PERSIANN_CDR"""
    }

    select_estacoes_vizinhas = '\n'+'\n'.join([f'        ,vl_precipitacao_vizinha_{i+1}' for i in range(n_estacoes_vizinhas)])
    select_satelites = ''.join([satelite_query for satelite,satelite_query in satelite_columns_str_dict.items() if satelite in satelite_list]).replace('    ','  ')
    
    query = f"""
    SELECT
        id_estacao
        ,dt_medicao
        ,MONTH(CAST(dt_medicao AS DATE)) AS dt_mes_medicao
        ,YEAR(CAST(dt_medicao AS DATE)) AS dt_ano_medicao
        ,CASE
            WHEN MONTH(CAST(dt_medicao AS DATE)) IN (11,12,1,2,3) THEN 1
            ELSE 0
        END AS fl_mes_chuvoso
        ,lat_estacao
        ,lon_estacao
        ,vl_temperatura_minima
        ,vl_temperatura_media
        ,vl_temperatura_maxima
        ,vl_umidade_relativa_minima
        ,vl_umidade_relativa_media{select_estacoes_vizinhas}
        ,vl_precipitacao{select_satelites}
    FROM r2_ouro.ABT_satelite_all
    WHERE 1=1
        AND YEAR(CAST(dt_medicao AS DATE)) >= {min_year_training}
        AND vl_precipitacao>=0
        AND vl_precipitacao<={max_prec_filter}
    """

    return sql(query)

## Dados Faltantes

### tratar_dados_faltantes

In [ ]:
def tratar_dados_faltantes(df,col,metodo):
    new_df = df.copy()
    if metodo == 'drop':
        new_df = new_df.dropna(subset=[col])
    elif metodo == 'zero':
        new_df[col] = new_df[col].fillna(0)
    elif metodo == 'media_mensal':
        new_df[col] = new_df[col].fillna(new_df.groupby(['dt_mes_medicao','dt_ano_medicao'])[col].transform('mean'))
    elif metodo == 'media_anual':
        new_df[col] = new_df[col].fillna(new_df.groupby(['dt_ano_medicao'])[col].transform('mean'))
    elif metodo == 'media_historico':
        media_historico = new_df['vl_precipitacao'].mean()
        new_df[col] = new_df[col].fillna(media_historico)
    
    return new_df

### print_relatorio_missing

In [ ]:
def print_relatorio_missing(df,col,total_rows):
    missing_rows = len(df.loc[df[col].isna()])
    if missing_rows>0:
        print(f'{col} - Missing: {missing_rows}/{total_rows} ({missing_rows*100/total_rows:.2f})')
    return 

### tratar_missing_estacoes_vizinhas

In [55]:
def tratar_missing_estacoes_vizinhas(abt,metodo,print_relatorio=True):

    metodos_possiveis = ['drop','zero','media_mensal','media_anual','media_historico']

    if not metodo in metodos_possiveis:
        raise ValueError(f"Parâmetro '{metodo}' não permitido para o argumento 'metodo'. Utilizar um dos valores possíveis: {', '.join(metodos_possiveis)}.")

    if print_relatorio:
        print("Relatório ESTAÇÕES VIZINHAS: Valores faltantes pré-tratamento")
        total_rows = len(abt)
        for col in abt.columns:
            print_relatorio_missing(abt,col,total_rows)
        print("------------------------------------------------------------------")

    precipitacao_vizinhas_columns = [col for col in abt.columns if 'vl_precipitacao_vizinha' in col]

    new_abt = abt.copy()
    for col in precipitacao_vizinhas_columns:
        new_abt = tratar_dados_faltantes(new_abt,col,metodo)

    if print_relatorio:
        print("Relatório ESTAÇÕES VIZINHAS: Valores faltantes pós-tratamento")
        total_rows = len(new_abt)
        for col in new_abt.columns:
            print_relatorio_missing(new_abt,col,total_rows)
        print("------------------------------------------------------------------")

    return new_abt

### tratar_missing_estacao_base

In [54]:
def tratar_missing_estacao_base(abt,metodo,print_relatorio=True):

    metodos_possiveis = ['drop','zero','media_mensal','media_anual','media_historico']

    if not metodo in metodos_possiveis:
        raise ValueError(f"Parâmetro '{metodo}' não permitido para o argumento 'metodo'. Utilizar um dos valores possíveis: {', '.join(metodos_possiveis)}.")
    
    if print_relatorio:
        print("Relatório ESTAÇÃO BASE: Valores faltantes pré-tratamento")
        total_rows = len(abt)
        for col in abt.columns:
            print_relatorio_missing(abt,col,total_rows)
        print("------------------------------------------------------------------")


    estacao_base_columns = [col for col in abt.columns if col.startswith('vl') and ('temperatura' in col or 'umidade' in col) and not 'POWER' in col]
    
    new_abt = abt.copy()
    for col in estacao_base_columns:
        new_abt = tratar_dados_faltantes(new_abt,col,metodo)

    if print_relatorio:
        print("Relatório ESTAÇÃO BASE: Valores faltantes pós-tratamento")
        total_rows = len(new_abt)
        for col in new_abt.columns:
            print_relatorio_missing(new_abt,col,total_rows)
        print("------------------------------------------------------------------")
            
    return new_abt

In [75]:
abt = import_abt(satelite_list=['CPC','GPM_Final_Run','POWER'],min_year_training=2016)
abt = tratar_missing_estacoes_vizinhas(abt,metodo='media_anual')
abt = tratar_missing_estacao_base(abt,metodo='media_historico')

Relatório ESTAÇÕES VIZINHAS: Valores faltantes pré-tratamento
vl_temperatura_minima - Missing: 16/303407 (0.01)
vl_temperatura_media - Missing: 124/303407 (0.04)
vl_umidade_relativa_minima - Missing: 1022/303407 (0.34)
vl_umidade_relativa_media - Missing: 793/303407 (0.26)
vl_precipitacao_vizinha_1 - Missing: 85160/303407 (28.07)
vl_precipitacao_vizinha_2 - Missing: 109491/303407 (36.09)
vl_precipitacao_vizinha_3 - Missing: 91738/303407 (30.24)
vl_precipitacao_vizinha_4 - Missing: 85818/303407 (28.28)
vl_precipitacao_GPM_Final_Run - Missing: 6973/303407 (2.30)
------------------------------------------------------------------
Relatório ESTAÇÕES VIZINHAS: Valores faltantes pós-tratamento
vl_temperatura_minima - Missing: 16/303407 (0.01)
vl_temperatura_media - Missing: 124/303407 (0.04)
vl_umidade_relativa_minima - Missing: 1022/303407 (0.34)
vl_umidade_relativa_media - Missing: 793/303407 (0.26)
vl_precipitacao_GPM_Final_Run - Missing: 6973/303407 (2.30)
--------------------------------

In [37]:
[col for col in abt.columns if col.startswith('vl') and ('temperatura' in col or 'umidade' in col) and not 'POWER' in col]

['vl_temperatura_minima',
 'vl_temperatura_media',
 'vl_temperatura_maxima',
 'vl_umidade_relativa_minima',
 'vl_umidade_relativa_media']

In [48]:
abt = tratar_missing_estacoes_vizinhas(abt,metodo='media_anual').dropna()

Relatório: Valores faltantes pré-tratamento
vl_temperatura_minima - Missing: 1426/901993 (0.16)
vl_temperatura_media - Missing: 35630/901993 (3.95)
vl_temperatura_maxima - Missing: 2018/901993 (0.22)
vl_umidade_relativa_minima - Missing: 281949/901993 (31.26)
vl_umidade_relativa_media - Missing: 36150/901993 (4.01)
vl_precipitacao_vizinha_1 - Missing: 427130/901993 (47.35)
vl_precipitacao_vizinha_2 - Missing: 446367/901993 (49.49)
vl_precipitacao_vizinha_3 - Missing: 414062/901993 (45.91)
vl_precipitacao_vizinha_4 - Missing: 456615/901993 (50.62)
vl_precipitacao_GPM_Final_Run - Missing: 190356/901993 (21.10)
vl_precipitacao_CPC - Missing: 78/901993 (0.01)
------------------------------------------------------------------
Relatório: Valores faltantes pós-tratamento
vl_temperatura_minima - Missing: 1426/901993 (0.16)
vl_temperatura_media - Missing: 35630/901993 (3.95)
vl_temperatura_maxima - Missing: 2018/901993 (0.22)
vl_umidade_relativa_minima - Missing: 281949/901993 (31.26)
vl_umidad

,id_estacao,dt_medicao,dt_mes_medicao,dt_ano_medicao,fl_mes_chuvoso,lat_estacao,lon_estacao,vl_temperatura_minima,vl_temperatura_media,vl_temperatura_maxima,...,vl_temperatura_media_2m_POWER,vl_temperatura_orvalho_2m_POWER,vl_direcao_vento_2m_POWER,vl_direcao_vento_10m_POWER,vl_maxima_vento_2m_POWER,vl_ws2m_POWER,vl_maxima_vento_10m_POWER,vl_vento_10m_POWER,vl_precipitacao_GPM_Final_Run,vl_precipitacao_CPC
0,30928,2003-04-09,4,2003,0,-21.7425,-41.3328,22.9,26.40,31.1,...,24.66,21.26,134.04,134.30,3.44,1.92,4.74,2.85,0.4,0.06
1,30928,2003-04-19,4,2003,0,-21.7425,-41.3328,20.9,26.18,31.0,...,23.77,17.55,49.65,49.28,3.77,2.50,5.08,3.79,0.0,0.00
2,30928,2003-05-13,5,2003,0,-21.7425,-41.3328,19.7,23.18,28.8,...,22.14,18.20,132.74,133.63,2.56,1.41,3.48,2.24,0.1,3.13
3,30928,2003-05-23,5,2003,0,-21.7425,-41.3328,20.3,25.42,32.4,...,24.47,16.40,315.86,314.63,3.86,2.47,5.34,3.83,0.0,0.33
4,30928,2003-05-24,5,2003,0,-21.7425,-41.3328,22.7,25.94,30.4,...,24.09,18.28,246.53,246.06,3.74,2.32,5.32,3.35,2.9,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901798,31029,2008-12-07,12,2008,1,-23.4800,-47.4267,15.3,22.32,31.5,...,21.40,14.00,130.31,130.31,3.37,2.34,5.12,3.55,0.0,0.00
901799,31029,2011-11-11,11,2011,1,-23.4800,-47.4267,21.0,24.30,31.3,...,24.01,16.90,122.31,122.44,3.32,1.79,5.09,2.78,2.4,0.25
901804,31029,2010-06-06,6,2010,0,-23.4800,-47.4267,7.2,12.36,21.0,...,11.76,7.71,190.81,190.44,1.59,1.13,3.01,2.13,0.0,0.00
901805,31029,2007-06-23,6,2007,0,-23.4800,-47.4267,12.7,18.70,27.0,...,16.37,12.51,209.56,209.69,4.58,2.14,6.89,3.76,0.0,0.00


# Desbalanceamento